## Packages for performing the optimization

In [ ]:
import numpy as np
import statistics
from statistics import mean, pvariance
import scipy
from scipy.optimize import minimize,NonlinearConstraint

## Packages for performing map manipulations

In [ ]:
from __future__ import print_function
from pixell import enmap,utils
import numpy as np
import matplotlib.pyplot as plt
from pixell import enplot
import os,sys
import urllib.request

## Uploading the maps from the cluster

In [ ]:
filename1 = "/moto/hill/projects/actpol/SO_Sims_LAMBDA/d56/SO_skymaps_deep56/so_skymap_deep56_30GHz.fits"
filename2 = "/moto/hill/projects/actpol/SO_Sims_LAMBDA/d56/SO_skymaps_deep56/so_skymap_deep56_44GHz.fits"
filename3 = "/moto/hill/projects/actpol/SO_Sims_LAMBDA/d56/SO_skymaps_deep56/so_skymap_deep56_70GHz.fits"
filename4 = "/moto/hill/projects/actpol/SO_Sims_LAMBDA/d56/SO_skymaps_deep56/so_skymap_deep56_100GHz.fits"
filename5 = "/moto/hill/projects/actpol/SO_Sims_LAMBDA/d56/SO_skymaps_deep56/so_skymap_deep56_143GHz.fits"
filename6 = "/moto/hill/projects/actpol/SO_Sims_LAMBDA/d56/SO_skymaps_deep56/so_skymap_deep56_145GHz.fits"
filename7 = "/moto/hill/projects/actpol/SO_Sims_LAMBDA/d56/SO_skymaps_deep56/so_skymap_deep56_217GHz.fits"
filename8 = "/moto/hill/projects/actpol/SO_Sims_LAMBDA/d56/SO_skymaps_deep56/so_skymap_deep56_353GHz.fits"

imap30 = enmap.read_map(filename1)
imap44 = enmap.read_map(filename2)
imap70 = enmap.read_map(filename3)
imap100 = enmap.read_map(filename4)
imap143 = enmap.read_map(filename5)
imap145 = enmap.read_map(filename6)
imap217 = enmap.read_map(filename7)
imap353 = enmap.read_map(filename8)

print("Map shape and dtype (same for all the maps):")
print(imap44.shape, imap44.dtype)

fmap30 = np.ndarray.flatten(imap30)
fmap44 = np.ndarray.flatten(imap44)
fmap70 = np.ndarray.flatten(imap70)
fmap100 = np.ndarray.flatten(imap100)
fmap143 = np.ndarray.flatten(imap143)
fmap145 = np.ndarray.flatten(imap145)
fmap217 = np.ndarray.flatten(imap217)
fmap353 = np.ndarray.flatten(imap353)

print("Flattened map:")
print(fmap44)

## Defining ΔT

In [ ]:
n_map = 8
n_pix = 2182*9455
map_array = [fmap30,fmap44,fmap70,fmap100,fmap143,fmap145,fmap217,fmap353]

## Defining the R matrix

In [ ]:
r = np.zeros([n_map,n_map])

for i in range(n_map):
    for j in range(n_map):
        r[i][j] = np.sum((map_array[i] * map_array[j])) / n_pix
        
print(r)

In [ ]:
b = np.zeros([n_map,n_map,n_map])

for i in range(n_map):
    for j in range(n_map):
        for k in range(n_map):
            b[i][j][k] = np.sum(map_array[i] * map_array[j] * map_array[k]) / n_pix
            
print(b)

## Optimizing the variance

In [ ]:
w0 = np.array([.125,.125,.125,.125,.125,.125,.125,.125])
a = np.ones(len(w0))

#defines a lambda function that returns dot product of weights and array of ones
fun = lambda w: np.dot(w,a)

print("The sum of all the weights is:")
print(fun(w0))

#nonlinear constraint that implements the function and sets the constraint bounds
con = NonlinearConstraint(fun,1,1)

#defines a lambda function that implements the variance equation
var = lambda w: np.dot(np.dot(w,r),w)

print("The variance is:")
print(var(w0))

#minimization method ()
mini = minimize(var,w0,method='SLSQP',constraints=con)
weights_var = mini.x
print(mini.x)

print()
print(var(mini.x))

print()
print(fun(mini.x))

## Optimizing the skewness

In [ ]:
w0 = np.array([.125,.125,.125,.125,.125,.125,.125,.125])
a = np.ones(len(w0))

#defines a lambda function that returns dot product of weights and array of ones
fun = lambda w: np.dot(w,a)

print("The sum of all the weights is:")
print(fun(w0))

#nonlinear constraint that implements the function and sets the constraint bounds
con = NonlinearConstraint(fun,1,1)

#defines a lambda function that implements the variance equation
skew = lambda w: abs(np.dot(np.dot(np.dot(w,b),w), w))

print()
print("The skewness is:")
print(skew(w0))

#minimization method ()
mini = minimize(skew,w0,method='SLSQP',constraints=con)
weights_skew = mini.x

print()
print("The solution array for the weights is:")
print(mini.x)

print()
print(skew(mini.x))

print()
print("The sum of the weights in the solution array is:")
print(fun(mini.x))

In [ ]:
imap_array = np.array([imap30,imap44,imap70,imap100,imap143,imap145,imap217,imap353])

#yp_var = np.tensordot(weights_var,imap_array, axes=0)
#yp_skew = np.tensordot(weights_skew, imap_array, axes=0)

yp_var = imap30.copy()
yp_var *= 0.

for i in range(n_map):
    yp_var += imap_array[i]*weights_var[i]
    
print(yp_var)

yp_skew = imap30.copy()
yp_skew *= 0

for i in range(n_map):
    yp_skew += imap_array[i]*weights_skew[i]

print(yp_skew)

In [ ]:
plots = enplot.plot(yp_var, range=300,mask=0)

def eshow(x,**kwargs): enplot.show(enplot.plot(x,**kwargs))
eshow(yp_var)

In [ ]:
plots = enplot.plot(yp_skew, range=300,mask=0)

def eshow(x,**kwargs): enplot.show(enplot.plot(x,**kwargs))
eshow(yp_skew)

In [ ]:
plots = enplot.plot(yp_var-yp_skew, range=300,mask=0)

def eshow(x,**kwargs): enplot.show(enplot.plot(x,**kwargs))
eshow(yp_var-yp_skew)

## Comparison of the Variance and Skewness

In [ ]:
freq = np.array([30,44,70,100,143,145,217,353])

diff = weights_var - (weights_skew / 10)
plt.plot(freq, abs(diff))
plt.show()

## Plot of the Sets of Weights

In [ ]:
freq = np.array([30,44,70,100,143,145,217,353])

plot = plt.plot(freq, weights_var,'b-',freq,weights_skew/10,'r')
plt.xlabel('Frequency')
plt.ylabel('Variance and Skewness')
plt.legend(['Variance','Skewness / 10'])
plt.show()
print("Note that the skewness weights are reduced to a tenth of their original value to better compare to the variance weights.")